# Emmigration data

In [8]:
import pandas as pd
from IPython.display import Markdown;
import numpy


## emigration_country_born.tsv

In [9]:
url = "../../Data/emigration/emigration_country_born.tsv"
table_raw = pd.read_csv(url,sep='\t')
year_range = range(2008,2024,1)

In [10]:

age = []
agedef = []
c_birth = []
unit = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row[0].split(","))

    age.append(title_split[1]) 
    agedef.append(title_split[2])
    c_birth.append(title_split[3])
    sex.append(title_split[5])
    country.append(title_split[6])
    
    

table_raw["age"]= age
table_raw["agedef"] = agedef
table_raw["birth_country"] = c_birth
table_raw["sex"]=sex
table_raw["country"]=country

table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)
display(table_raw.head())

/tmp/ipykernel_782089/3369307096.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title_split = (row[0].split(","))


,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,...,2019,2020,2021,2022,2023,age,agedef,birth_country,sex,country
0,0,0,:,0,0,:,0,1,0,1,...,0,0,0,0,0,TOTAL,COMPLET,AD,F,AT
1,:,:,2 b,2,0,0,2,1,0,1,...,0,0,1,1,0,TOTAL,COMPLET,AD,F,BE
2,:,:,:,:,0 bp,0 p,0 p,0 p,0,0 p,...,0 p,0 p,0,0,0,TOTAL,COMPLET,AD,F,BG
3,:,:,:,0 b,0,0,1,4,4,1,...,4,3,:,:,:,TOTAL,COMPLET,AD,F,CH
4,:,:,:,:,:,:,:,:,:,:,...,:,0,0,0,0,TOTAL,COMPLET,AD,F,CZ


In [11]:
#rename country codes
country_codes = pd.read_csv("../../Data/country-region-codes.csv")
new_columns = {}

#numeric columns
for column in table_raw.columns:
    if column.strip().isnumeric():
        new_columns[column] = int(column.strip())
    else:
        new_columns[column]=column
        
table = table_raw.rename(columns=new_columns)
table.rename(columns={"country":"alpha-2"},inplace=True)

table = pd.merge(table,country_codes,on="alpha-2").drop(columns=["country-code","alpha-2"]).rename(columns={"name":"Country","alpha-3":"ISO3"})

#reordering

new_order = list(table.columns[-6:])+[i for i in year_range]# IMPORTANT to set -6 to number of non numeric columns

table = table[new_order]
display(table.head())

,age,agedef,birth_country,sex,Country,ISO3,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,AD,F,Austria,AUT,0,0,:,0,...,0,1,0,1,0,0,0,0,0,0
1,TOTAL,COMPLET,AD,F,Belgium,BEL,:,:,2 b,2,...,2,1,0,1,1,0,0,1,1,0
2,TOTAL,COMPLET,AD,F,Bulgaria,BGR,:,:,:,:,...,0 p,0 p,0,0 p,0 p,0 p,0 p,0,0,0
3,TOTAL,COMPLET,AD,F,Switzerland,CHE,:,:,:,0 b,...,1,4,4,1,3,4,3,:,:,:
4,TOTAL,COMPLET,AD,F,Czechia,CZE,:,:,:,:,...,:,:,:,:,:,:,0,0,0,0


#### Fixing values
Renaming : to NaN
fixing provisional data

In [12]:
replaced = table.copy()

#fix numeric values
for column in replaced.columns:
    replaced[column] = replaced[column].apply(str.strip)
    # print(type(replaced[column][0]))

replaced.replace(to_replace=":",value=numpy.nan,inplace=True)

#tags
tags = ["b","p","ep","e","be","bep","P"]

#I used Deepseek AI for this, it was quite complex
pattern = r'\b(?:' + '|'.join(tags) + r')\b'

# Apply the replacement to all columns (or specify specific columns)
df = replaced.apply(lambda x: x.str.replace(pattern, '', regex=True).str.strip())

#### Saving to file

In [14]:
path = "../../Data/emigration/emigration_country_born.csv"

display(df.head())
df.to_csv(path,index=False)

,age,agedef,birth_country,sex,Country,ISO3,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,AD,F,Austria,AUT,0,0,NaN,0,...,0,1,0,1,0,0,0,0,0,0
1,TOTAL,COMPLET,AD,F,Belgium,BEL,NaN,NaN,2,2,...,2,1,0,1,1,0,0,1,1,0
2,TOTAL,COMPLET,AD,F,Bulgaria,BGR,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,TOTAL,COMPLET,AD,F,Switzerland,CHE,NaN,NaN,NaN,0,...,1,4,4,1,3,4,3,NaN,NaN,NaN
4,TOTAL,COMPLET,AD,F,Czechia,CZE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
